In [1]:

from src.inserter import ChromaCollection, embedding_function
from src.text_splitter import load_from_zipfile


/home/desbian/cosas/deepy/codebase-rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
zip_path="itba-paw-2024a-04-0203e222159e.zip"

docs=load_from_zipfile(zip_path)


✓ Parseando: pom.xml (xml)
  → 1 fragmentos extraídos
✓ Parseando: pom.xml (xml)
  → 1 fragmentos extraídos
✓ Parseando: AppointmentService.java (java)
  → 2 fragmentos extraídos
✓ Parseando: PasswordRecoveryCodeDao.java (java)
  → 2 fragmentos extraídos
✓ Parseando: ImageService.java (java)
  → 2 fragmentos extraídos
✓ Parseando: UserService.java (java)
  → 2 fragmentos extraídos
✓ Parseando: ServiceDao.java (java)
  → 2 fragmentos extraídos
✓ Parseando: UserDao.java (java)
  → 2 fragmentos extraídos
✓ Parseando: ImageDao.java (java)
  → 2 fragmentos extraídos
✓ Parseando: ManageServiceService.java (java)
  → 2 fragmentos extraídos
✓ Parseando: RatingService.java (java)
  → 2 fragmentos extraídos
✓ Parseando: EmailService.java (java)
  → 2 fragmentos extraídos
✓ Parseando: UserVerificationService.java (java)
  → 2 fragmentos extraídos
✓ Parseando: AppointmentDao.java (java)
  → 2 fragmentos extraídos
✓ Parseando: BusinessService.java (java)
  → 2 fragmentos extraídos
✓ Parseando: Ques

/home/desbian/cosas/deepy/codebase-rag/lib/python3.11/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


  → 2 fragmentos extraídos
✓ Parseando: ServiceService.java (java)
  → 2 fragmentos extraídos
✓ Parseando: RatingDao.java (java)
  → 2 fragmentos extraídos
✓ Parseando: UserVerificationDao.java (java)
  → 2 fragmentos extraídos
✓ Parseando: pom.xml (xml)
  → 1 fragmentos extraídos
✓ Parseando: ValidImageFile.java (java)
  → 1 fragmentos extraídos
✓ Parseando: LocationFormatted.java (java)
  → 1 fragmentos extraídos
✓ Parseando: FieldsValueMatch.java (java)
  → 1 fragmentos extraídos
✓ Parseando: EmailRegisteredValidator.java (java)
  → 2 fragmentos extraídos
✓ Parseando: UsernameNotUsedValidator.java (java)
  → 2 fragmentos extraídos
✓ Parseando: ValidImageFileValidator.java (java)
  → 2 fragmentos extraídos
✓ Parseando: FieldsValueMatchValidator.java (java)
  → 2 fragmentos extraídos
✓ Parseando: EmailNotUsed.java (java)
  → 1 fragmentos extraídos
✓ Parseando: UsernameNotUsed.java (java)
  → 1 fragmentos extraídos
✓ Parseando: EmailRegistered.java (java)
  → 1 fragmentos extraídos
✓ P

In [3]:
chroma=ChromaCollection("repoRAG")
chroma.insert_docs(docs=docs)

chroma_collection=chroma.chroma_collection


# plots

In [18]:
pip install umap-learn numpy pandas nbformat plotly

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [4]:
import umap
import numpy as np
from tqdm import tqdm

embeddings = chroma_collection.get(include=['embeddings'])['embeddings']
umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(embeddings)

/home/desbian/cosas/deepy/codebase-rag/lib/python3.11/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [5]:
def project_embeddings(embeddings, umap_transform):
  """
  Projects high-dimensional embeddings into a 2D space using UMAP.

  Parameters:
  embeddings (numpy.ndarray): A 2D array-like object containing the embeddings to be transformed,
                              where each row represents an embedding vector.
  umap_transform (umap.UMAP): A pre-trained UMAP model to perform the transformation.

  Returns:
  numpy.ndarray: A 2D array where each row is a 2D embedding resulting from the UMAP transformation.
  """
  umap_embeddings = np.empty((len(embeddings),2))   # Mappeamos desde la longitud de nuestros embeddings a 2D

  for i, embedding in enumerate(tqdm(embeddings)):
      umap_embeddings[i] = umap_transform.transform([embedding])

  return umap_embeddings

In [6]:
projected_dataset_embeddings = project_embeddings(embeddings, umap_transform)

100%|██████████| 285/285 [00:09<00:00, 31.25it/s] 


In [7]:
import plotly.express as px
import pandas as pd

def plot_embeddings(projected_dataset_embeddings):
    """
    Plots 2D projected embeddings using Plotly Express.

    Parameters:
    projected_dataset_embeddings (numpy.ndarray): A 2D array containing the projected embeddings,
                                                  where each row represents a point in 2D space.
    """

    df_embeddings = pd.DataFrame(projected_dataset_embeddings, columns=['x', 'y'])

    fig = px.scatter(df_embeddings, x='x', y='y', title='Projected Embeddings')

    fig.update_layout(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        showlegend=False,
        title_x=0.5
    )

    fig.update_yaxes(scaleanchor="x", scaleratio=1)  # Maintain aspect ratio
    fig.show()

In [8]:
plot_embeddings(projected_dataset_embeddings)

In [9]:
import plotly.graph_objects as go
import pandas as pd

def plot_relevant_docs(projected_dataset_embeddings,
                       projected_query_embedding,
                       projected_retrieved_embeddings,
                       query, title, projected_augmented_embeddings=[]):
    """
    Plots the query, dataset, and retrieved documents in 2D embedding space using Plotly.

    Parameters:
    projected_dataset_embeddings (numpy.ndarray): 2D array of projected dataset embeddings.
    projected_query_embedding (numpy.ndarray): 2D array containing the projected query embedding.
    projected_retrieved_embeddings (numpy.ndarray): 2D array of projected embeddings for retrieved documents.
    query (str): The query text to be displayed on hover.
    title (str): Title of the plot.
    """
    # Convert data to DataFrame for easier plotting
    df_dataset = pd.DataFrame(projected_dataset_embeddings, columns=['x', 'y'])
    df_query = pd.DataFrame(projected_query_embedding, columns=['x', 'y'])
    df_query['text'] = query  # Add query text for hover information
    df_retrieved = pd.DataFrame(projected_retrieved_embeddings, columns=['x', 'y'])
    df_augmented = pd.DataFrame(projected_augmented_embeddings, columns=['x', 'y'])

    # Create a scatter plot for the dataset points
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df_dataset['x'],
        y=df_dataset['y'],
        mode='markers',
        marker=dict(size=10, color='gray'),
        name='Dataset Embeddings'
    ))

    fig.add_trace(go.Scatter(
        x=df_query['x'],
        y=df_query['y'],
        mode='markers',
        marker=dict(size=15, symbol='x', color='red'),
        name='Query Embedding',
        hovertext=[query],  # Add hover text
        hoverinfo='text'
    ))

    fig.add_trace(go.Scatter(
        x=df_retrieved['x'],
        y=df_retrieved['y'],
        mode='markers',
        marker=dict(size=12, symbol='circle-open', line=dict(color='green', width=2)),
        name='Retrieved Embeddings'
    ))

    fig.add_trace(go.Scatter(
        x=df_augmented['x'],
        y=df_augmented['y'],
        mode='markers',
        marker=dict(size=15, symbol='x', color='pink'),
        name='Augmented Embeddings'
    ))

    fig.update_layout(
        title=title,
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        showlegend=True,
        title_x=0.5
    )

    fig.show()

In [10]:
def project_and_plot_relevant_docs(query, title):

  retrieved_documents, results = chroma.retrieve_k_similar_docs(query, k=5)
  query_embedding = embedding_function([query])[0]
  retrieved_embeddings = results['embeddings'][0]

  projected_query_embedding = project_embeddings([query_embedding], umap_transform)
  projected_retrieved_embeddings = project_embeddings(retrieved_embeddings, umap_transform)

  plot_relevant_docs(projected_dataset_embeddings, projected_query_embedding, projected_retrieved_embeddings, query, title)
  return retrieved_documents

In [14]:
query="Que dependencias hay en los pom.xml"
project_and_plot_relevant_docs(query, title=query)

100%|██████████| 5/5 [00:00<00:00, 58.00it/s]


['<?xml version="1.0" encoding="UTF-8" standalone="no"?>\n<project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 http://maven.apache.org/xsd/maven-4.0.0.xsd">\n  <modelVersion>4.0.0</modelVersion>\n  <groupId>ar.edu.itba.paw</groupId>\n  <artifactId>paw2024</artifactId>\n  <version>1.0-SNAPSHOT</version>\n  <packaging>pom</packaging>\n  <name>paw2024</name>\n\n  <properties>\n    <project.build.sourceEncoding>UTF-8</project.build.sourceEncoding>\n    <maven.compiler.source>21</maven.compiler.source>\n    <maven.compiler.target>21</maven.compiler.target>\n    <common-fileupload.version>1.3.1</common-fileupload.version>\n    <spring.version>5.3.33</spring.version>\n    <servlet-api.version>2.5</servlet-api.version>\n    <jstl.version>1.2</jstl.version>\n    <postgresql.version>42.2.5</postgresql.version>\n    <mockito.version>2.25.1</mockito.version>\n    <junit.version>4.13.2</junit.v